In [2]:
# 03 - Création Masques BDTOPO
# Crée l'ensemble des masques indiquant la présence ou non d'un bâtiment
# En entrée :
#   - dft = bdtopo.pkl : base BDTOPO des polygones
#   - dfi = bdortho.pkl : dataframe de l'ensemble des images 1000x1000
# En sortie :
#   - masques enregistrés dans le répertoire choisi
#   - dfi mis à jour avec le nom des fichiers des masques et le nombre de polygones par image

In [7]:
# Paths globaux
HOME = '/Users/u600141/OneDrive - La Francaise des Jeux/Data/constructions/'
PATHS_BDTOPO_PICKLE = 'images/BDTOPO/pickle/'

# Si RGB
PATHS_BDTOPO_MASKS = 'images/BDORTHO_RGB/masks_1000x1000/'
PATHS_BDORTHO_PICKLE = HOME + 'images/BDORTHO_RGB/pickle/'

# Si IRC
"""
PATHS_BDTOPO_MASKS = 'images/BDORTHO_IRC/masks_1000x1000/'
PATHS_BDORTHO_PICKLE = HOME + 'images/BDORTHO_IRC/pickle/'
"""

# Définitions globales
K = 1000 # Taille des images


In [8]:
# Imports
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as sh
import glob
import matplotlib.pyplot as plt
import cv2
import rasterio.features
from affine import Affine

In [13]:
# Chargement des méta-données BDTOPO
dft = pd.read_pickle(PATHS_BDTOPO_PICKLE + 'dft.pkl')
print (f'dft chargé, taille : {len(dft)}')

# Chargement des méta-données BDORTHO
dfi = pd.read_pickle(PATHS_BDORTHO_PICKLE + 'dfi.pkl')
print (f'dfi chargé, taille : {len(dfi)}')


dft chargé, taille : 694950
dfi chargé, taille : 11875


In [10]:
# notmalizePolygons() permet de normaliser, si nécessaire, les coordonnées des polygones :
#   - entre 0 et 1
#   - origine en haut à gauche (d'où reverseY)
#   - abs() est utilisée pour supprimer les erreurs d'arrondi négatif vers 0
# Cette normalisation est notamment nécessaire pour les modèles YOLO
def normalizePolygon(xy, xrange, yrange, reverseY=False):
    if (reverseY==True):
        ret = abs((xy - [xrange[0], yrange[1]]) * [1/(xrange[1]-xrange[0]), 1/(yrange[0]-yrange[1])])
    else:
        ret = abs((xy - [xrange[0], yrange[0]]) * [1/(xrange[1]-xrange[0]), 1/(yrange[1]-yrange[0])])
    return ret

# getIntersectingPolygons() retourne les polygones en intersection avec le rectangle,
#    de surface > minArea, normalisés si nécessaire et avec les coordonnées de haut en bas si nécessaire
def getIntersectingPolygons(df, xrange, yrange, minArea=0, normalize=False, reverseY=False):
    rect_poly = sh.Polygon([(xrange[0], yrange[0]),
                            (xrange[1], yrange[0]),
                            (xrange[1], yrange[1]),
                            (xrange[0], yrange[1])])
    intersect = df['geometry'].values.intersection(rect_poly)
    intersect = intersect[~ intersect.is_empty]
    if (normalize==True):
        intersect = gpd.GeoSeries(sh.transform(intersect, lambda p : normalizePolygon(p, xrange, yrange, reverseY)))
    if (minArea > 0):
        intersect = intersect[intersect.area >= minArea]
    return intersect

In [ ]:
# Création des masques BDTOPO et ajout de leur path dans le dataframe des méta-données des images (dfi)
# On ajoute également le nombde de polygones de chaque image

mask_filenames = []
nb_polygons = []

for index, row in dfi.iterrows():
    xrange, yrange = row['xrange'], row['yrange']
    intersect = getIntersectingPolygons(dft, xrange=xrange, yrange=yrange)
    if (len(intersect)==0):
        img = np.zeros((K, K))
    else:
        aff = Affine((xrange[1]-xrange[0])/K, 0.0, xrange[0], 0.0, -(yrange[1]-yrange[0])/K, yrange[1])
        img = rasterio.features.rasterize(intersect, out_shape=(K, K), transform=aff) * 255
    nb_polygons.append(len(intersect))
    filename = PATHS_BDTOPO_MASKS + 'mask-' + row['filename'].split('/')[3].replace('.jpg', '.png')
    cv2.imwrite(filename, img)
    mask_filenames.append(filename)

dfi['mask_filename'] = mask_filenames
dfi['NumberOfPolygons'] = nb_polygons

In [6]:
# Mise à jour du dataframe au format pickle .pkl

path = PATHS_BDORTHO_PICKLE + 'dfi.pkl'
dfi.to_pickle(path)
print (f'DataFrame sauvegardé sous : {path}')


DataFrame sauvegardé sous : /Users/u600141/OneDrive - La Francaise des Jeux/Data/constructions/images/BDORTHO_IRC/pickle/df_images_meta.pkl
